In [1]:
import gymnasium as gym
from tqdm import tqdm
import sys
import os
import torch
import re
import json

rocket_dir = os.path.split(os.path.abspath(os.curdir))[:-1]
rocket_src_dir = os.path.join(*rocket_dir, 'src')
rocket_dir = os.path.join(*rocket_dir)

print(f'Appending to sys.path:', [rocket_dir, rocket_src_dir])

sys.path.append(rocket_dir)
sys.path.append(rocket_src_dir)

from src.environments.frozen_env import make_frozen_env, FrozenActions
from src.agents.Mistral_agent import MistralAgent

Appending to sys.path: ['/home/davide/rocket', '/home/davide/rocket/src']


/home/davide/miniconda3/envs/idefics/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment FrozenLakeText-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [2]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

agent = MistralAgent("mistralai/Mistral-7B-Instruct-v0.2",
                        action_enum=FrozenActions,
                         is_lora=True, 
                         padding_side="right", 
                         gradient_ckpt=False)
agent.critic.to('cuda')

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

trainable params: 27,262,976 || all params: 7,268,995,072 || trainable%: 0.3750583915652433


Sequential(
  (0): Linear(in_features=4096, out_features=1024, bias=True)
  (1): ReLU()
  (2): Linear(in_features=1024, out_features=512, bias=True)
  (3): ReLU()
  (4): Linear(in_features=512, out_features=1, bias=True)
)

In [3]:
runs_directory = 'runs_directory_prova'
envs = gym.vector.SyncVectorEnv(
            [make_frozen_env(
                runs_directory,
                area=8, # 8x8,
                seed=1234 + i + 1,
                size=(512, 512),
                is_slippery=False,
                fixed_orientation=True,
                save_video=False,
                save_video_every=20,
                save_stats=False,
                fov=3
            )
            for i in range(1)
            ])

In [13]:
import numpy as np

obs, info = envs.reset(seed=42)
count = 0

# get random observation by stepping the environment
while True:
    action = envs.action_space.sample()
    obs, reward, done, truncated, info = envs.step(action)
    
    if done:
        envs.reset()
    elif np.random.rand() >= 0.8:
        _, contexts, _ = agent.create_prompt_for_action_and_value(text_description=info['obs'])
        context = contexts[0]
        print('_____________CONTEXT PROMPT______________')
        print(context)
        print('_____________CONTUINUATION______________')
        continuation = agent.generate_continuation(context)
        print(continuation)
        print('_____________EXTRACT ACTION______________')

        action_list = [a.value for a in agent.action_enum]
        fixtemplate_action_choices = [f"Based on the information provided, the best action would be to {action}" for action in action_list]
        action_choices, match_found = agent.find_action_in_sentence_and_create_action_choices(continuation, action_list, fixtemplate_action_choices)
        print('match found:', match_found)
        for ac in action_choices:
            print(ac)
        count += 1
        print('========================================================\n\n')
        
    if count > 3:
        break







_____________CONTEXT PROMPT______________
<s>[INST] You are an agent in a survival 2D game. You took action move east.
You see:
- a trap 1 steps to your east.
- a trap 3 steps to your south-west.
- the goal 12 steps to your south-east.
 What's the next best action? You must avoid the traps! Choose exactly one from [move west, move south, move east, move north] [/INST]
_____________CONTUINUATION______________
Based on the information provided, the next best action would be to move north to avoid both the trap 1 step to the east and the trap 3 steps to the south-west. The goal being 12 steps to the south-east is not a concern at the moment as it is further away and reaching it would involve passing through the trap first.</s>
_____________EXTRACT ACTION______________
match found: 1
Based on the information provided, the next best action would be to move north to avoid both the trap 1 step to the east and the trap 3 steps to the south-west. The goal being 12 steps to the south-east is not